In [1]:
import os
import sys
import warnings
import numpy as np
from pandas import read_csv
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import traceback

def myprint(string, clear=False):
    if clear:
        sys.stdout.write("\033[F")
        sys.stdout.write("\033[K")
    print(string)

warnings.filterwarnings("ignore")

# Tangos setup
os.environ['TANGOS_DB_CONNECTION'] = '/home/bk639/data_base/CDM_all.db'
os.environ['TANGOS_SIMULATION_FOLDER'] = '/home/bk639/data/CDM_z0'
os.environ['TANGOS_PROPERTY_MODULES'] = 'mytangosproperty'
sys.path.append('/home/bk639/mytangosproperty')
import tangos
import mytangosproperty

# Functions from original code
def sersic(r, mueff, reff, n):
    return mueff + 2.5*(0.868*n-0.142)*((r/reff)**(1./n) - 1)

def func(x, a, c, d):
    return a*np.exp(-c*x)+d

def correlation(ell, sb):
    if not isinstance(ell, np.ndarray): ell = np.array(ell)
    if not isinstance(sb, np.ndarray): sb = np.array(sb)
    n = np.nansum((ell-np.nanmean(ell)) * (sb-np.nanmean(sb)))
    d1 = np.nansum((ell-np.nanmean(ell))**2)
    d2 = np.nansum((sb-np.nanmean(sb))**2)
    return n/np.sqrt(d1*d2)

def myround(x, base=5):
    return base * round(x/base)

def randomorientation(base=30):
    phi = np.degrees(np.random.uniform(0, np.pi*2))
    y = int(myround(phi, base=base))
    theta = np.degrees(np.arccos(np.random.uniform(-1, 1)))
    x = int(myround(theta, base=base))
    if x > 165:
        x = x % 180
        y = (180-y)
        if y < 0:
            y = y + 360
    if y == 360:
        y = 0
    return(f'x{x:03d}y{y:03d}')

# Hubble parameter for mass conversion
h = 0.6776931508813172

# Initialize dictionaries to store data
SimInfo = {}
ProfData, ProjData, MorphData, ShapeData, MassData = {}, {}, {}, {}, {}
Masses = {}
nhalo = 0

# Get simulations from Tangos
sims = tangos.all_simulations()
sims = sims[0:1]
print(f"Found {len(sims)} simulations")


Found 1 simulations


In [2]:


# Process each simulation
for sim in sims:
    sim_name = str(sim.basename)
    print(f"Processing simulation {sim_name}")

    # Initialize dictionaries for this simulation
    SimInfo[sim_name] = {'halos': [], 'goodhalos': []}
    ProfData[sim_name] = {}
    ProjData[sim_name] = {}
    MorphData[sim_name] = {}
    ShapeData[sim_name] = {}
    MassData[sim_name] = {}
    Masses[sim_name] = {}

    # Get the latest snapshot
    if len(sim.timesteps) == 1:
        timestep = sim.timesteps[0]
    else:
        timestep = sim.timesteps[-1]

    halos = timestep.halos
    halos = halos[0:2]  # Limit to the first halo for testing

    # Process each halo
    for halo in halos:
        try:
            hid = halo.halo_number
            hid_str = str(hid)

            # Check if halo has enough star particles
            if 'n_star' in halo.keys() and halo['n_star'][0] < 4000:
                print(f"Skipping halo {hid} - too few star particles")
                continue

            print(f"Processing halo {hid}")

            # Initialize data structures for this halo
            ProfData[sim_name][hid_str] = {}
            ProjData[sim_name][hid_str] = {}
            ShapeData[sim_name][hid_str] = {}
            MorphData[sim_name][hid_str] = {}

            # Get half-light radius
            if 'Rhalf' in halo.keys():
                Rhalf = halo['Rhalf']
                # Create the face-on orientation entry if it doesn't exist
                if 'x000y000' not in ProfData[sim_name][hid_str]:
                    ProfData[sim_name][hid_str]['x000y000'] = {}
                ProfData[sim_name][hid_str]['x000y000']['Rhalf'] = Rhalf

            # Get images, orientations, and profile data
            if all(k in halo.keys() for k in ['halo_images', 'image_orientations', 'profile_sb_v', 'profile_v_lum_den',
                                             'profile_rbins', 'profile_lum_den', 'profile_mags_v', 'profile_binarea']):

                # Get the data from Tangos
                halo_images = halo['halo_images']
                image_orientations = halo['image_orientations']
                profile_sb_v = halo['profile_sb_v']
                profile_v_lum_den = halo['profile_v_lum_den']
                profile_rbins = halo['profile_rbins']
                profile_lum_den = halo['profile_lum_den']
                profile_mags_v = halo['profile_mags_v']
                profile_binarea = halo['profile_binarea']
                reff = halo['image_reffs']
                #verify all data is present
    
                    

                # Process each orientation
                for i, orientation in enumerate(image_orientations):
                    # Convert orientation angle to string format (e.g., 'x030y060')
                    print(orientation)
                    angle_str = orientation

                    ProfData[sim_name][hid_str][angle_str]['sb,v'] = profile_sb_v[i]
                    ProfData[sim_name][hid_str][angle_str]['v_lum_den'] = profile_v_lum_den[i]
                    ProfData[sim_name][hid_str][angle_str]['rbins'] = profile_rbins[i]
                    ProfData[sim_name][hid_str][angle_str]['lum_den'] = profile_lum_den[i]
                    ProfData[sim_name][hid_str][angle_str]['mags,v'] = profile_mags_v[i]
                    ProfData[sim_name][hid_str][angle_str]['binarea'] = profile_binarea[i]
                    ProfData[sim_name][hid_str][angle_str]['Reff'] = reff[i]

                    # Calculate central surface brightness (Sigma0)
                    if 'mags,v' in ProfData[sim_name][hid_str][angle_str] and 'binarea' in ProfData[sim_name][hid_str][angle_str]:
                        mags_v = ProfData[sim_name][hid_str][angle_str]['mags,v']
                        binarea = ProfData[sim_name][hid_str][angle_str]['binarea']
                        if len(mags_v) > 0 and len(binarea) > 0:
                            mag0 = mags_v[0]
                            area0 = binarea[0]
                            Sigma0 = (10**(0.4*(4.8-mag0)))/area0
                            ProfData[sim_name][hid_str][angle_str]['Sigma0'] = Sigma0
                    break

            # Get ellipse data (using the 2nd element from ellipses list)
            ellipses = halo['ellipses']
            if len(ellipses) > 1:
                ellipse_data = ellipses[1]  # Use the second element (index 1)

                # Get the orientations
                orientations = halo['image_orientations']

                # Process each orientation
                for i, orientation in enumerate(orientations):
                    angle_str = orientation

                    # Initialize dictionaries if needed
                    if angle_str not in ProjData[sim_name][hid_str]:
                        ProjData[sim_name][hid_str][angle_str] = {}
                    if angle_str not in ShapeData[sim_name][hid_str]:
                        ShapeData[sim_name][hid_str][angle_str] = {}

                    # Store ellipse data if available for this orientation
                    if i < len(ellipse_data) and 'b/a' in ellipse_data[i]:
                        b_over_a = ellipse_data[i]['b/a']
                        ProjData[sim_name][hid_str][angle_str]['b/a'] = b_over_a
                        ShapeData[sim_name][hid_str][angle_str]['b/a'] = b_over_a

            # Get 3D shape data
            ba_values = halo['ba_s']
            ca_values = halo['ca_s']

            # Create smooth interpolation functions
            MorphData[sim_name][hid_str]['ba_smooth'] = halo.calculate('ba_s_smoothed')
            MorphData[sim_name][hid_str]['ca_smooth'] = halo.calculate('ca_s_smoothed')

            # Get mass data (applying hubble parameter)
            if 'Mvir' in halo.keys():
                MassData[sim_name][hid_str] = halo['Mvir'] * h

            if 'Mstar' in halo.keys():
                Masses[sim_name][hid_str] = {'Mstar': halo['Mstar'] * h}

            # Add halo to lists
            SimInfo[sim_name]['halos'].append(hid)
            SimInfo[sim_name]['goodhalos'].append(hid)

            # Increment halo counter
            nhalo += 1

        except Exception as e:
            print(f"Error processing halo {hid if 'hid' in locals() else 'unknown'}: {str(e)}")
            traceback.print_exc()
            

print(f"Loaded data for {nhalo} halos across {len(sims)} simulations")

Processing simulation cptmarvel.cosmo25cmb.4096g5HbwK1BH
Processing halo 1
Image orientations: ['x000y000', 'x000y030', 'x000y060', 'x000y090', 'x000y120', 'x000y150', 'x000y180', 'x000y210', 'x000y240', 'x000y270', 'x000y300', 'x000y330', 'x030y000', 'x030y030', 'x030y060', 'x030y090', 'x030y120', 'x030y150', 'x030y180', 'x030y210', 'x030y240', 'x030y270', 'x030y300', 'x030y330', 'x060y000', 'x060y030', 'x060y060', 'x060y090', 'x060y120', 'x060y150', 'x060y180', 'x060y210', 'x060y240', 'x060y270', 'x060y300', 'x060y330', 'x090y000', 'x090y030', 'x090y060', 'x090y090', 'x090y120', 'x090y150', 'x090y180', 'x090y210', 'x090y240', 'x090y270', 'x090y300', 'x090y330', 'x120y000', 'x120y030', 'x120y060', 'x120y090', 'x120y120', 'x120y150', 'x120y180', 'x120y210', 'x120y240', 'x120y270', 'x120y300', 'x120y330', 'x150y000', 'x150y030', 'x150y060', 'x150y090', 'x150y120', 'x150y150', 'x150y180', 'x150y210', 'x150y240', 'x150y270', 'x150y300', 'x150y330']
Profile SB: [array([24.82899913, 24.9956

Traceback (most recent call last):
  File "/tmp/ipykernel_1944633/1432722496.py", line 103, in <module>
    ellipses = halo['ellipses']
               ~~~~^^^^^^^^^^^^
  File "/home/bk639/miniconda3/envs/pynbody_beta/lib/python3.12/site-packages/tangos/core/halo.py", line 194, in __getitem__
    return self.get_data(key)
           ^^^^^^^^^^^^^^^^^^
  File "/home/bk639/miniconda3/envs/pynbody_beta/lib/python3.12/site-packages/tangos/core/halo.py", line 219, in get_data
    return_data = self.get_objects(key, getters)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bk639/miniconda3/envs/pynbody_beta/lib/python3.12/site-packages/tangos/core/halo.py", line 236, in get_objects
    key_id = get_dict_id(key, session=session)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bk639/miniconda3/envs/pynbody_beta/lib/python3.12/site-packages/tangos/core/dictionary.py", line 45, in get_dict_id
    return _dict_id[text]
           ~~~~~~~~^^^^^^
KeyError: 'ellipses'
Trace

In [6]:

# Calculate M/L ratios
for sim in SimInfo:
    for halo in ProfData[sim]:
        if halo not in MassData[sim]:
            continue

        for angle in ProfData[sim][halo]:

            # Get effective radius
            Reff = ProfData[sim][halo]['x000y000']['Reff']

            # Get magnitude and radius bins
            mags = ProfData[sim][halo][angle]['mags,v']
            rbins = ProfData[sim][halo][angle]['rbins']

            # Find index closest to Reff
            indeff = np.argmin(np.abs(Reff - rbins))

            # Calculate luminosity
            lums = 10**(0.4*(4.8-mags))
            lumeff = np.sum(lums[:indeff+1]) if indeff < len(lums) else np.sum(lums)

            # Calculate M/L
            ProfData[sim][halo][angle]['M/L'] = MassData[sim][halo] / lumeff


xu_folder = '/home/bk639/MorphologyMeasurements'
# Xu & Randall 2020 data
LGall = [-0.287, 0.058, '#7B0B01', 'All LG Dsphs']
LGbright = [-0.353, 0.059, '#FFCC99', 'LG Dsphs\n'+r'$M/L<100M_\odot/L_\odot$']
LGdim = [0.509, 0.309, '#339AFE', 'LG Dsphs\n'+r'$M/L>100M_\odot/L_\odot$']
FIRE = [-0.322, 0.213, '#411B52', 'FIRE Simulations']
FIRE_Data = read_csv(xu_folder+'/Data/BasicData/FIRE_Data.csv')
FIRE_x = FIRE_Data['X'].tolist()
FIRE_y = FIRE_Data['Y'].tolist()

# Analysis parameters
n_itr = int(1e5)
xbins = np.linspace(-1, 1, 500)
Xu = False
EnvSplit = False
Scatter = True
DimSplit = False
IndCor = False

# Scatter Plot Analysis
if Scatter:
    Xu_Dim = read_csv(xu_folder+'/Data/BasicData/Xu_Scatter_Dim.csv')
    DimX = Xu_Dim['X'].tolist()
    DimY = Xu_Dim['Y'].tolist()
    Xu_Bright = read_csv(xu_folder+'/Data/BasicData/Xu_Scatter_Bright.csv')
    BrightX = Xu_Bright['X'].tolist()
    BrightY = Xu_Bright['Y'].tolist()

    xbx, xby, xbxl, xbxu, xbyl, xbyu = [], [], [], [], [], []
    xdx, xdy, xdxl, xdxu, xdyl, xdyu = [], [], [], [], [], []
    i = 0
    while i < len(DimX) - 4:
        xdx.append(DimX[i])
        xdxl.append(DimX[i] - DimX[i+1])
        xdxu.append(DimX[i+2] - DimX[i])
        xdy.append(DimY[i])
        xdyl.append(DimY[i] - DimY[i+3])
        xdyu.append(DimY[i+4] - DimY[i])
        i += 5
    i = 0
    while i < len(BrightX) - 4:
        xbx.append(BrightX[i])
        xbxl.append(BrightX[i] - BrightX[i+1])
        xbxu.append(BrightX[i+2] - BrightX[i])
        xby.append(BrightY[i])
        xbyl.append(BrightY[i] - BrightY[i+3])
        xbyu.append(BrightY[i+4] - BrightY[i])
        i += 5

    x, y, xl, xu, yl, yu = [], [], [], [], [], []
    bx, by, bxl, bxu, byl, byu = [], [], [], [], [], []
    dx, dy, dxl, dxu, dyl, dyu = [], [], [], [], [], []

    for sim in SimInfo:
        for halo in SimInfo[sim]['halos']:
            halo = str(halo)
            if halo not in ProfData[sim]:
                continue

            ells, sbs = [], []
            ellsb, sbsb = [], []
            ellsd, sbsd = [], []

            for angle in ProfData[sim][halo]:
                if ('b/a' not in ProjData[sim][halo][angle] or
                    'Sigma0' not in ProfData[sim][halo][angle]):
                    continue

                ells.append(1 - ProjData[sim][halo][angle]['b/a'])
                sbs.append(ProfData[sim][halo][angle]['Sigma0'])

                if 'x000y000' in ProfData[sim][halo] and 'M/L' in ProfData[sim][halo]['x000y000']:
                    if ProfData[sim][halo]['x000y000']['M/L'] < 100:
                        ellsb.append(1 - ProjData[sim][halo][angle]['b/a'])
                        sbsb.append(ProfData[sim][halo][angle]['Sigma0'])
                    else:
                        ellsd.append(1 - ProjData[sim][halo][angle]['b/a'])
                        sbsd.append(ProfData[sim][halo][angle]['Sigma0'])

            if len(ells) > 0 and len(sbs) > 0:
                x.append(np.nanmean(sbs))
                y.append(np.nanmean(ells))
                xl.append(np.nanmean(sbs) - np.min(sbs))
                xu.append(np.max(sbs) - np.nanmean(sbs))
                yl.append(np.nanmean(ells) - np.min(ells))
                yu.append(np.max(ells) - np.nanmean(ells))

                if 'x000y000' in ProfData[sim][halo] and 'M/L' in ProfData[sim][halo]['x000y000']:
                    if ProfData[sim][halo]['x000y000']['M/L'] < 100 and len(ellsb) > 0:
                        bx.append(np.nanmean(sbsb))
                        by.append(np.nanmean(ellsb))
                        bxl.append(np.nanmean(sbsb) - np.min(sbsb))
                        bxu.append(np.max(sbsb) - np.nanmean(sbsb))
                        byl.append(np.nanmean(ellsb) - np.min(ellsb))
                        byu.append(np.max(ellsb) - np.nanmean(ellsb))
                    elif len(ellsd) > 0:
                        dx.append(np.nanmean(sbsd))
                        dy.append(np.nanmean(ellsd))
                        dxl.append(np.nanmean(sbsd) - np.min(sbsd))
                        dxu.append(np.max(sbsd) - np.nanmean(sbsd))
                        dyl.append(np.nanmean(ellsd) - np.min(ellsd))
                        dyu.append(np.max(ellsd) - np.nanmean(ellsd))

    # Generate plots
    f, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.set_ylim([0, 1])
    ax.set_xlim([2e-2, 1e3])
    ax.semilogx()
    ax.set_yticks([0, .2, .4, .6, .8])
    ax.tick_params(which='both', labelsize=15)
    ax.set_xlabel(r'Surface Brightness (L$_\odot$/pc$^2$)', fontsize=20)
    ax.set_ylabel(r'Ellipticity $(1-b/a)$', fontsize=20)

    ax.errorbar(xbx, xby, xerr=[xbxl, xbxu], yerr=[xbyl, xbyu], c='#FFCC99', fmt='none')
    ax.scatter(xbx, xby, c='#FFCC99', label=r'LG Dsphs $M/L<100M_\odot/L_\odot$')

    ax.errorbar(xdx, xdy, xerr=[xdxl, xdxu], yerr=[xdyl, xdyu], c='#339AFE', fmt='none')
    ax.scatter(xdx, xdy, c='#339AFE', label=r'LG Dsphs $M/L>100M_\odot/L_\odot$')

    ax.errorbar(bx, by, xerr=[bxl, bxu], yerr=[byl, byu], c='goldenrod', fmt='none', zorder=0)
    ax.scatter(bx, by, c='goldenrod', zorder=1, label=r'Sim $M/L<100M_\odot/L_\odot$')

    ax.errorbar(dx, dy, xerr=[dxl, dxu], yerr=[dyl, dyu], c='navy', fmt='none', zorder=0)
    ax.scatter(dx, dy, c='navy', zorder=1, label=r'Sim $M/L>100M_\odot/L_\odot$')

    ax.legend(loc='upper right', prop={'size': 12})
    f.savefig('../../Figures/CorrelationTesting/Ell_vs_SB.png', bbox_inches='tight', pad_inches=.1)
    plt.close()

    # Plot without error bars
    f, ax = plt.subplots(1, 1, figsize=(8, 6))
    ax.set_ylim([0, 1])
    ax.set_xlim([2e-2, 1e3])
    ax.semilogx()
    ax.set_yticks([0, .2, .4, .6, .8])
    ax.tick_params(which='both', labelsize=15)
    ax.set_xlabel(r'Surface Brightness (L$_\odot$/pc$^2$)', fontsize=20)
    ax.set_ylabel(r'Ellipticity $(1-b/a)$', fontsize=20)

    ax.scatter(xbx, xby, c='#FFCC99', label=r'LG Dsphs $M/L<100M_\odot/L_\odot$')
    ax.scatter(xdx, xdy, c='#339AFE', label=r'LG Dsphs $M/L>100M_\odot/L_\odot$')
    ax.scatter(bx, by, c='k', zorder=1, label=r'Sim $M/L<100M_\odot/L_\odot$')
    ax.scatter(dx, dy, c='r', zorder=1, label=r'Sim $M/L>100M_\odot/L_\odot$')

    ax.legend(loc='upper right', prop={'size': 12})
    f.savefig('../../Figures/CorrelationTesting/Ell_vs_SB.NoBar.png', bbox_inches='tight', pad_inches=.1)
    plt.close()

# Define functions for other analyses (Xu, EnvSplit, DimSplit, IndCor)
# These would follow a similar pattern to the Scatter analysis above
# Implement as needed based on which flags are set to True

print(f"Analysis complete. Total halos processed: {nhalo}")

Processing simulation cptmarvel.cosmo25cmb.4096g5HbwK1BH
Processing halo 2
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
x000y000
Error storing profile data for cptmarvel.cosmo25cmb.4096g5HbwK1BH-2-x000y000: object of type 'float' has no len()


Traceback (most recent call last):
  File "/tmp/ipykernel_482023/136953391.py", line 107, in <module>
    if len(mags_v) > 0 and len(binarea) > 0:
       ^^^^^^^^^^^
TypeError: object of type 'float' has no len()


Loaded data for 4 halos across 1 simulations
Error calculating M/L for cptmarvel.cosmo25cmb.4096g5HbwK1BH-2-x000y000: object of type 'float' has no len()


FileNotFoundError: [Errno 2] No such file or directory: '/home/bk639/MorphologyMeasurementsData/BasicData/Xu_Scatter_Dim.csv'